# 環境整理

In [1]:
!pip install japanize-matplotlib
!pip install optuna
!pip install optuna-integration[lightgbm]
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 24.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for japanize-matplotlib: filename=japanize_matplotlib-1.1.3-py3-none-any.whl size=4120257 sha256=b29011fddc1a2a86a4e41fd8438f29b3445d4e478bd2764ebe5f3d7e07696388
  Stored in directory: /root/.cache/pip/wheels/da/a1/71/b8faeb93276fed10edffcca20746f1ef6f8d9e071eee8425fc
Successfully built japanize-matplotlib
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.7/242.7 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.9/98.9 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.1 MB/s eta 0:00:00


In [2]:
import pandas as pd
import plotly.express as px
import lightgbm as lgb
import numpy as np
import missingno as msno
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, StackingClassifier, GradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.metrics import roc_auc_score, accuracy_score, recall_score, roc_curve, auc, precision_score, f1_score, confusion_matrix
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from scipy.spatial import distance
from scipy.optimize import differential_evolution
import seaborn as sns
import japanize_matplotlib
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split, KFold, RandomizedSearchCV, GridSearchCV, cross_val_score
from lightgbm import LGBMClassifier
from imblearn.over_sampling import SMOTE
import optuna
from optuna.integration import lightgbm as lgb_optuna
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)

In [100]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [192]:
# データの読み込み
train = pd.read_csv('drive/MyDrive/UTTC/train_data.csv')
test = pd.read_csv('drive/MyDrive/UTTC/test_data.csv')
submission = pd.read_csv('drive/MyDrive/UTTC/submission.csv')

# 特徴量エンジニアリング

In [193]:
train['Churn'].replace({'No': 0, 'Yes': 1}, inplace=True)

In [194]:
train['SeniorCitizen'].replace({'No': 0, 'Yes': 1}, inplace=True)
test['SeniorCitizen'].replace({'No': 0, 'Yes': 1}, inplace=True)

In [195]:
# trainデータフレームにフラグを追加
train['No_Internet_Flag'] = train['InternetService'] == 'No'
train['OnlineSecurity_TechSupport_Flag'] = (train['OnlineSecurity'] == 'Yes') & (train['TechSupport'] == 'Yes')
train['Dependents_MultipleLines_Flag'] = (train['Dependents'] == 'Yes') & (train['MultipleLines'] == 'Yes')
train['MultipleLines_OnlineSecurity_Flag'] = (train['MultipleLines'] == 'Yes') & (train['OnlineSecurity'] == 'Yes')

# testデータフレームにフラグを追加
test['No_Internet_Flag'] = test['InternetService'] == 'No'
test['OnlineSecurity_TechSupport_Flag'] = (test['OnlineSecurity'] == 'Yes') & (test['TechSupport'] == 'Yes')
test['Dependents_MultipleLines_Flag'] = (test['Dependents'] == 'Yes') & (test['MultipleLines'] == 'Yes')
test['MultipleLines_OnlineSecurity_Flag'] = (test['MultipleLines'] == 'Yes') & (test['OnlineSecurity'] == 'Yes')

In [196]:
#削除するカラム
columns_to_drop = ['customerID']
train = train.drop(columns=columns_to_drop)
test = test.drop(columns=columns_to_drop)

In [ ]:
#新規顧客のフラグ
train["is_new_customer"] = (train["tenure"] < 12).astype(int)
test["is_new_customer"] = (test["tenure"] < 12).astype(int)

In [198]:
#契約残り月数を追加
train['month_remainder'] = np.select(
    [
        train['Contract'] == 'Month-to-month',
        train['Contract'] == 'One year',
        train['Contract'] == 'Two year'
    ],
    [
        1,
        12 - (train['tenure'] % 12),
        24 - (train['tenure'] % 24)
    ],
    default=np.nan  # 念のため他の契約形式があれば
)

test['month_remainder'] = np.select(
    [
        test['Contract'] == 'Month-to-month',
        test['Contract'] == 'One year',
        test['Contract'] == 'Two year'
    ],
    [
        1,
        12 - (test['tenure'] % 12),
        24 - (test['tenure'] % 24)
    ],
    default=np.nan
)

In [199]:
#その他の新変数
service_cols = ['PhoneService', 'MultipleLines', 'OnlineSecurity', 'OnlineBackup',
                'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies']

train['NumServices'] = train[service_cols].apply(lambda x: sum(x == 'Yes'), axis=1)
test['NumServices'] = test[service_cols].apply(lambda x: sum(x == 'Yes'), axis=1)

train['RiskyContract'] = ((train['Contract'] == 'Month-to-month') & (train['PaperlessBilling'] == 'Yes')).astype(int)
test['RiskyContract'] = ((test['Contract'] == 'Month-to-month') & (test['PaperlessBilling'] == 'Yes')).astype(int)

train['RiskyContract2'] = ((train['Contract'] == 'Month-to-month') & (train['PhoneService'] == 'No')).astype(int)
test['RiskyContract2'] = ((test['Contract'] == 'Month-to-month') & (test['PhoneService'] == 'No')).astype(int)

train["SeniorPaperless"] = (train["SeniorCitizen"] == 1) & (train["PaperlessBilling"] == "Yes")
test["SeniorPaperless"] = (test["SeniorCitizen"] == 1) & (test["PaperlessBilling"] == "Yes")

train["Is_Monthly_Electronic"] = (train["Contract"] == "Month-to-month") & (train["PaymentMethod"] == "Electronic check")
test["Is_Monthly_Electronic"] = (test["Contract"] == "Month-to-month") & (test["PaymentMethod"] == "Electronic check")

In [200]:
# 空白文字（" "）や文字列が混じっていても、数値に変換できるようにする
train['TotalCharges'] = pd.to_numeric(train['TotalCharges'], errors='coerce')
test['TotalCharges'] = pd.to_numeric(test['TotalCharges'], errors='coerce')
train['TotalCharges'] = train['TotalCharges'].fillna(0)
test['TotalCharges'] = test['TotalCharges'].fillna(0)

train['ExpectedGap'] = train['MonthlyCharges'] * train['tenure'] - train['TotalCharges']
test['ExpectedGap'] = test['MonthlyCharges'] * test['tenure'] - test['TotalCharges']

train["MonthlyCharges"] = np.log1p(train["MonthlyCharges"])
test["MonthlyCharges"] = np.log1p(test["MonthlyCharges"])

In [201]:
#予め、重回帰分析を行ったところ、各サービスの単価が判明。単価でカテゴリデータをエンコーディング。
def encode_services(df):
    df = df.copy()

    df['InternetService'] = df['InternetService'].map({
        'Fiber optic': 50,
        'DSL': 25
    }).fillna(0)

    df['PhoneService'] = df['PhoneService'].map({'Yes': 20}).fillna(0)
    df['StreamingMovies'] = df['StreamingMovies'].map({'Yes': 10}).fillna(0)
    df['StreamingTV'] = df['StreamingTV'].map({'Yes': 10}).fillna(0)
    df['OnlineSecurity'] = df['OnlineSecurity'].map({'Yes': 5}).fillna(0)
    df['TechSupport'] = df['TechSupport'].map({'Yes': 5}).fillna(0)
    df['MultipleLines'] = df['MultipleLines'].map({'Yes': 5}).fillna(0)
    df['DeviceProtection'] = df['DeviceProtection'].map({'Yes': 5}).fillna(0)
    df['OnlineBackup'] = df['OnlineBackup'].map({'Yes': 5}).fillna(0)

    return df

# エンコーディング適用
train = encode_services(train)
test = encode_services(test)

In [202]:
binary_cols = ['Partner', 'Dependents', 'PaperlessBilling']

# 変換マップ
yes_no_map = {'Yes': 1, 'No': 0}

# train, test 両方に変換を適用
for col in binary_cols:
    if col in train.columns:
        train[col] = train[col].map(yes_no_map)
    if col in test.columns:
        test[col] = test[col].map(yes_no_map)

train['gender'].replace({'Male': 0, 'Female': 1}, inplace=True)
test['gender'].replace({'Male': 0, 'Female': 1}, inplace=True)

In [203]:
onehot_cols = [
    'PaymentMethod', 'Contract'
]

# train, test 両方に同じカテゴリを持たせるため、まず結合
train['__is_train__'] = 1
test['__is_train__'] = 0
combined = pd.concat([train, test], axis=0)

# One-Hot Encoding（NaNは無視されます）
combined = pd.get_dummies(combined, columns=onehot_cols, drop_first=False)

# 再び train/test に分割
train = combined[combined['__is_train__'] == 1].drop(columns='__is_train__').reset_index(drop=True)
test = combined[combined['__is_train__'] == 0].drop(columns='__is_train__').reset_index(drop=True)
test = test.drop("Churn", axis=1)

In [204]:
X_train = train.drop("Churn", axis=1).copy()
X_test = test
X_all = pd.concat([X_train, X_test], axis=0)

scaler = StandardScaler()
X_all_scaled = scaler.fit_transform(X_all)

X_train_scaled = X_all_scaled[:len(train)]
X_test_scaled = X_all_scaled[len(train):]

#共分散行列の逆行列を作成（trainデータに基づく）
mean_vec = np.mean(X_train_scaled, axis=0)
cov_matrix = np.cov(X_train_scaled, rowvar=False)
inv_cov_matrix = np.linalg.pinv(cov_matrix)

#マハラノビス距離を計算することで、異常値を可視化
mahal_train = [distance.mahalanobis(row, mean_vec, inv_cov_matrix) for row in X_train_scaled]
mahal_test = [distance.mahalanobis(row, mean_vec, inv_cov_matrix) for row in X_test_scaled]

#結果を元データに追加
train["Mahalanobis"] = mahal_train
test["Mahalanobis"] = mahal_test

# 機械学習

In [ ]:
# 特徴量と目的変数の分離
y_train = train["Churn"].values
X_train = train.drop("Churn", axis=1).values
sc = StandardScaler()
X_train_std = sc.fit_transform(X_train)

# ====== train にスタッキング用のモデル出力列を追加 ======
kf = KFold(n_splits=5, shuffle=True, random_state=42)
train_meta_features = {
    "model_mlp": np.zeros(len(train)),
    "model_gb": np.zeros(len(train))
}

for train_idx, valid_idx in kf.split(X_train_std):
    X_tr, X_val = X_train_std[train_idx], X_train_std[valid_idx]
    y_tr = y_train[train_idx]
    model_mlp = MLPClassifier(
        hidden_layer_sizes=(64, 32),  # 隠れ層の構成
        max_iter=500,
        alpha=0.001,                  # 正則化
        random_state=0,
        early_stopping=True,
        verbose=False
    )
    model_gb = GradientBoostingClassifier(
        n_estimators=200,
        learning_rate=0.05,
        max_depth=5,
        subsample=0.8,
        random_state=0
    )

    model_mlp.fit(X_tr, y_tr)
    model_gb.fit(X_tr, y_tr)

    train_meta_features["model_mlp"][valid_idx] = model_mlp.predict_proba(X_val)[:, 1]
    train_meta_features["model_gb"][valid_idx] = model_gb.predict_proba(X_val)[:, 1]

# カラムとして追加
for name in train_meta_features:
    train[name] = train_meta_features[name]

# ====== test にも各モデルの確率出力を追加 ======
X_test_std = sc.transform(test.values)

test["model_mlp"] = model_mlp.predict_proba(X_test_std)[:, 1]
test["model_gb"] = model_gb.predict_proba(X_test_std)[:, 1]

In [ ]:
# データ準備
X = train.drop("Churn", axis=1).values
y = train["Churn"].values

# 標準化
sc = StandardScaler()
X_std = sc.fit_transform(X)

# クロスバリデーション設定
kf = KFold(n_splits=2, shuffle=True, random_state=0)

# メタ特徴量と特徴量重要度を初期化
train_valid_lgb = np.zeros(y.shape)
feature_importances = np.zeros(X_std.shape[1])

# 各foldでモデル学習
for fold, (train_index, valid_index) in enumerate(kf.split(X_std)):
    X_train, X_valid = X_std[train_index], X_std[valid_index]
    y_train, y_valid = y[train_index], y[valid_index]

    # モデル定義（不均衡対策 + ハイパーパラメータ調整）
    model_lgb = LGBMClassifier(
        random_state=0,
        class_weight='balanced',
        n_estimators=200,
        learning_rate=0.05,
        num_leaves=31,
        max_depth=-1
    )
    model_lgb.fit(X_train, y_train)

    # 検証用の確率予測
    y_prob = model_lgb.predict_proba(X_valid)[:, 1]
    train_valid_lgb[valid_index] = y_prob

    # 特徴量重要度を加算
    feature_importances += model_lgb.feature_importances_

# ======== しきい値最適化 =============
thresholds = np.arange(0.1, 0.9, 0.01)
best_f1 = 0
best_thresh = 0.5

for t in thresholds:
    preds = (train_valid_lgb > t).astype(int)
    score = f1_score(y, preds)
    if score > best_f1:
        best_f1 = score
        best_thresh = t

# 最終予測とF1スコア
final_preds = (train_valid_lgb > best_thresh).astype(int)
print(f"Best Threshold: {best_thresh}")
print(f"Optimized Train F1 Score: {round(best_f1, 5)}")

[LightGBM] [Info] Number of positive: 1343, number of negative: 3727
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002819 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1730
[LightGBM] [Info] Number of data points in the train set: 5070, number of used features: 39
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000
[LightGBM] [Info] Number of positive: 1364, number of negative: 3706
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002619 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1730
[LightGBM] [Info] Number of data points in the train set: 5070, number of used features: 39
[LightGBM] [Info] [binar

In [ ]:
# 標準化（Logistic Regressionでは必須）
sc = StandardScaler()
X_std = sc.fit_transform(X)

# クロスバリデーション設定
kf = KFold(n_splits=19, shuffle=True, random_state=0)

# メタ特徴量
train_valid_lr = np.zeros(y.shape)

# 各foldでモデル学習
for fold, (train_index, valid_index) in enumerate(kf.split(X_std)):
    X_train, X_valid = X_std[train_index], X_std[valid_index]
    y_train, y_valid = y[train_index], y[valid_index]

    model_lr = LogisticRegression(
        penalty='l2',              # L2正則化（標準的）
        solver='liblinear',       # 小規模データに強い
        class_weight='balanced',  # 不均衡対策
        random_state=0
    )
    model_lr.fit(X_train, y_train)

    # 確率予測
    y_prob = model_lr.predict_proba(X_valid)[:, 1]
    train_valid_lr[valid_index] = y_prob

# ======== しきい値最適化 =============
thresholds = np.arange(0.1, 0.9, 0.01)
best_f1 = 0
best_thresh = 0.5

for t in thresholds:
    preds = (train_valid_lr > t).astype(int)
    score = f1_score(y, preds)
    if score > best_f1:
        best_f1 = score
        best_thresh = t

# 最終予測とF1スコア
final_preds = (train_valid_lr > best_thresh).astype(int)
print(f"Best Threshold: {best_thresh}")
print(f"Optimized Train F1 Score: {round(best_f1, 5)}")

Best Threshold: 0.5599999999999997
Optimized Train F1 Score: 0.6297


In [ ]:
# 標準化（Random Forestは非線形モデルなので不要だが、LightGBMと比較のためにそのまま）
sc = StandardScaler()
X_std = sc.fit_transform(X)

# クロスバリデーション設定
kf = KFold(n_splits=16, shuffle=True, random_state=0)

# メタ特徴量と特徴量重要度を初期化
train_valid_rf = np.zeros(y.shape)
feature_importances = np.zeros(X_std.shape[1])

# 各foldでモデル学習
for fold, (train_index, valid_index) in enumerate(kf.split(X_std)):
    X_train, X_valid = X_std[train_index], X_std[valid_index]
    y_train, y_valid = y[train_index], y[valid_index]

    model_rf = RandomForestClassifier(
        n_estimators=200,
        max_depth=10,
        class_weight='balanced',  # 不均衡データ対策
        random_state=0,
        n_jobs=-1  # 並列処理
    )
    model_rf.fit(X_train, y_train)

    # 検証用の確率予測
    y_prob = model_rf.predict_proba(X_valid)[:, 1]
    train_valid_rf[valid_index] = y_prob

    # 特徴量重要度を加算
    feature_importances += model_rf.feature_importances_

# ======== しきい値最適化 =============
thresholds = np.arange(0.1, 0.9, 0.01)
best_f1 = 0
best_thresh = 0.5

for t in thresholds:
    preds = (train_valid_rf > t).astype(int)
    score = f1_score(y, preds)
    if score > best_f1:
        best_f1 = score
        best_thresh = t

# 最終予測とF1スコア
final_preds = (train_valid_rf > best_thresh).astype(int)
print(f"Best Threshold: {best_thresh}")
print(f"Optimized Train F1 Score: {round(best_f1, 5)}")

Best Threshold: 0.4099999999999998
Optimized Train F1 Score: 0.62609


In [ ]:
# 標準化（CatBoostは本来不要ですが、LightGBMと比較用に残しています）
sc = StandardScaler()
X_std = sc.fit_transform(X)

# クロスバリデーション設定
kf = KFold(n_splits=5, shuffle=True, random_state=0)

# メタ特徴量と特徴量重要度を初期化
train_valid_cb = np.zeros(y.shape)
feature_importances = np.zeros(X_std.shape[1])

# class weight 計算（不均衡データ対策）
class_weights = [1, (len(y) - sum(y)) / sum(y)]

# 各foldでモデル学習
for fold, (train_index, valid_index) in enumerate(kf.split(X_std)):
    X_train, X_valid = X_std[train_index], X_std[valid_index]
    y_train, y_valid = y[train_index], y[valid_index]

    model_cb = CatBoostClassifier(
        iterations=200,
        learning_rate=0.05,
        depth=6,
        random_seed=0,
        verbose=0,
        class_weights=class_weights
    )
    model_cb.fit(X_train, y_train)

    # 検証用の確率予測
    y_prob = model_cb.predict_proba(X_valid)[:, 1]
    train_valid_cb[valid_index] = y_prob

    # 特徴量重要度を加算
    feature_importances += model_cb.get_feature_importance()

# ======== しきい値最適化 =============
thresholds = np.arange(0.1, 0.9, 0.01)
best_f1 = 0
best_thresh = 0.5

for t in thresholds:
    preds = (train_valid_cb > t).astype(int)
    score = f1_score(y, preds)
    if score > best_f1:
        best_f1 = score
        best_thresh = t

# 最終予測とF1スコア
final_preds = (train_valid_cb > best_thresh).astype(int)
print(f"Best Threshold: {best_thresh}")
print(f"Optimized Train F1 Score: {round(best_f1, 5)}")

Best Threshold: 0.5399999999999998
Optimized Train F1 Score: 0.62431


In [210]:
train_valids = [train_valid_lgb, train_valid_lr, train_valid_rf,
                train_valid_cb]

# ======== 最適化関数定義：重み + 閾値 (0.1～0.9) を同時最適化 ========
def global_objective(params):
    weights = np.array(params[:-1])
    weights = weights / weights.sum()  # 正規化
    threshold = params[-1]

    # 加重平均
    blended = np.average(train_valids, axis=0, weights=weights)
    preds = (blended >= threshold).astype(int)
    score = f1_score(y, preds)
    return -score  # minimize用に負

# ======== 最適化設定（重み7つ + 閾値1つ = 8次元）========
bounds = [(0, 1)] * 4 + [(0.1, 0.9)]  # 閾値は0.1～0.9と制限

result = differential_evolution(global_objective, bounds, seed=42)
best_params = result.x
best_weights = best_params[:-1]
best_weights /= best_weights.sum()  # 正規化
best_thresh = best_params[-1]

print("Best weights:", best_weights)
print("Best threshold:", round(best_thresh, 4))
print("Train F1 Score:", round(-result.fun, 5))

# ========== FinalModel クラス定義 ==========
class FinalModel:
    def __init__(self, models, weights, scaler, threshold):
        self.models = models
        self.weights = weights / np.sum(weights)
        self.scaler = scaler
        self.threshold = threshold

    def predict_proba(self, X):
        X_std = self.scaler.transform(X)
        preds = [model.predict_proba(X_std)[:, 1] for model in self.models]
        return np.average(preds, axis=0, weights=self.weights)

    def predict(self, X):
        proba = self.predict_proba(X)
        return (proba >= self.threshold).astype(int)

# ========== モデル構築 ==========
models = [model_lgb, model_lr, model_rf, model_cb] # すでに学習済み
final_model = FinalModel(models=models, weights=best_weights, scaler=sc, threshold=best_thresh)

Best weights: [0.07652765 0.69651854 0.12376249 0.10319131]
Best threshold: 0.5313
Train F1 Score: 0.63226


# 予測結果の出力

In [211]:
y_pred_label = final_model.predict(test)
y_pred_label = np.where(y_pred_label == 1, "Yes", "No")

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


In [212]:
submission['Churn'] = y_pred_label

In [213]:
from google.colab import files
submission.to_csv('submission_prediction.csv', index=False)
files.download('submission_prediction.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>